In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing packages and helper functions

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# importing helper function
from helper_functions import create_tensorboard_callback, plot_loss_curves, pred_and_plot, unzip_data, walk_through_dir

--2021-11-14 12:28:06--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-11-14 12:28:06 (65.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



# Unzipping data and creating input Data

In [ ]:
unzip_data('/content/drive/MyDrive/Datasets/all_food_classes.zip')

## All class names

In [ ]:
# getting all calsses name
import pathlib

data_dir = pathlib.Path('all_food_classes/train')
classes = np.array(sorted([item.name for item in data_dir.glob("*")]))
class_names = classes[0:]
print(class_names)

['apple_pie' 'baby_back_ribs' 'baklava' 'beef_carpaccio' 'beef_tartare'
 'beet_salad' 'beignets' 'bibimbap' 'bread_pudding' 'breakfast_burrito'
 'bruschetta' 'caesar_salad' 'cannoli' 'caprese_salad' 'carrot_cake'
 'ceviche' 'cheese_plate' 'cheesecake' 'chicken_curry'
 'chicken_quesadilla' 'chicken_wings' 'chocolate_cake' 'chocolate_mousse'
 'churros' 'clam_chowder' 'club_sandwich' 'crab_cakes' 'creme_brulee'
 'croque_madame' 'cup_cakes' 'deviled_eggs' 'donuts' 'dumplings' 'edamame'
 'eggs_benedict' 'escargots' 'falafel' 'filet_mignon' 'fish_and_chips'
 'foie_gras' 'french_fries' 'french_onion_soup' 'french_toast'
 'fried_calamari' 'fried_rice' 'frozen_yogurt' 'garlic_bread' 'gnocchi'
 'greek_salad' 'grilled_cheese_sandwich' 'grilled_salmon' 'guacamole'
 'gyoza' 'hamburger' 'hot_and_sour_soup' 'hot_dog' 'huevos_rancheros'
 'hummus' 'ice_cream' 'lasagna' 'lobster_bisque' 'lobster_roll_sandwich'
 'macaroni_and_cheese' 'macarons' 'miso_soup' 'mussels' 'nachos'
 'omelette' 'onion_rings' 'oy

## Data preprocessing and Augmentation

In [ ]:
train_dir = '/content/all_food_classes/train'
test_dir = '/content/all_food_classes/test'

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 128

train_datagen = ImageDataGenerator(
    zoom_range = 0.2,
    rotation_range = 0.2,
    horizontal_flip = True,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    rescale = 1/255.,
)

test_datagen = ImageDataGenerator(rescale = 1/255.,)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size= IMG_SIZE,
    class_mode='categorical',
    batch_size= BATCH_SIZE,
    seed = 101,
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size= IMG_SIZE,
    class_mode='categorical',
    batch_size= BATCH_SIZE,
    seed = 101,
)

Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


## Training Callbacks

In [ ]:
train_callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=3,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5,
        patience=3, verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Datasets/SeeFood_models/model_V2', save_best_only=True),
]

# Model 

* **For model V2 we are using pretraind Xception model**

In [ ]:
base_model = tf.keras.applications.Xception(include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True

83697664/83683744 [==============================] - 1s 0us/step


In [ ]:
for layer in base_model.layers[:-22]:
  layer.trainable = False

In [ ]:
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)

model_V2 = tf.keras.Model(inputs, outputs)

In [ ]:
model_V2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 256)               524544    
                                                                 
 dense_3 (Dense)             (None, 101)               25957     
                                                                 
Total params: 21,411,981
Trainable params: 8,414,869
Non-trainable params: 12,997,112
_______________________________________

In [ ]:
model_V2.compile(
    loss = 'categorical_crossentropy',
    optimizer = Adam(),
    metrics = ['accuracy'],
)

history = model_V2.fit(
    train_data,
    epochs = 15,
    steps_per_epoch = len(train_data),
    validation_data = test_data,
    validation_steps = int(0.15 * len(test_data)),
    callbacks = train_callbacks,
)

Epoch 1/15
592/592 [==============================] - ETA: 0s - loss: 2.1809 - accuracy: 0.4636INFO:tensorflow:Assets written to: /content/drive/MyDrive/Datasets/SeeFood_models/model_V2/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


592/592 [==============================] - 1519s 3s/step - loss: 2.1809 - accuracy: 0.4636 - val_loss: 1.3316 - val_accuracy: 0.6517 - lr: 0.0010


/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


Epoch 2/15
592/592 [==============================] - ETA: 0s - loss: 1.5188 - accuracy: 0.6076INFO:tensorflow:Assets written to: /content/drive/MyDrive/Datasets/SeeFood_models/model_V2/assets


...

Epoch 5/15
592/592 [==============================] - 1481s 3s/step - loss: 1.0148 - accuracy: 0.7268 - val_loss: 1.0690 - val_accuracy: 0.7231 - lr: 0.0010
Epoch 6/15
592/592 [==============================] - ETA: 0s - loss: 0.9051 - accuracy: 0.7522INFO:tensorflow:Assets written to: /content/drive/MyDrive/Datasets/SeeFood_models/model_V2/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


592/592 [==============================] - 1468s 2s/step - loss: 0.9051 - accuracy: 0.7522 - val_loss: 0.9928 - val_accuracy: 0.7452 - lr: 0.0010


/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


Epoch 7/15
592/592 [==============================] - 1444s 2s/step - loss: 0.8266 - accuracy: 0.7706 - val_loss: 1.0372 - val_accuracy: 0.7400 - lr: 0.0010
Epoch 8/15
592/592 [==============================] - 1444s 2s/step - loss: 0.7397 - accuracy: 0.7916 - val_loss: 1.0748 - val_accuracy: 0.7236 - lr: 0.0010
Epoch 9/15
592/592 [==============================] - ETA: 0s - loss: 0.6662 - accuracy: 0.8107
Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
592/592 [==============================] - 1455s 2s/step - loss: 0.6662 - accuracy: 0.8107 - val_loss: 1.0618 - val_accuracy: 0.7414 - lr: 0.0010
Epoch 10/15
592/592 [==============================] - 1453s 2s/step - loss: 0.4234 - accuracy: 0.8768 - val_loss: 1.0446 - val_accuracy: 0.7589 - lr: 5.0000e-04
Epoch 11/15
592/592 [==============================] - 1429s 2s/step - loss: 0.3379 - accuracy: 0.8981 - val_loss: 1.1444 - val_accuracy: 0.7578 - lr: 5.0000e-04
Epoch 12/15
592/592 [===================

In [ ]:
model_V2.save('/content/drive/MyDrive/Datasets/SeeFood_models/SeeFood_General_Model_V2')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Datasets/SeeFood_models/SeeFood_General_Model_V2/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [ ]:
model_V2 = tf.keras.models.load_model('/content/drive/MyDrive/Datasets/SeeFood_models/SeeFood_General_Model_V2')

In [ ]:
model_V2.evaluate(test_data)

198/198 [==============================] - 253s 1s/step - loss: 1.4188 - accuracy: 0.7615


[1.4188088178634644, 0.7615445256233215]

# Converting model to tflite model

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/drive/MyDrive/Datasets/SeeFood_models/SeeFood_General_Model_V2')
tflite_model = converter.convert()

open("model_V2.tflite", "wb").write(tflite_model)

optimize="Speed"
if optimize=='Speed':
    converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]
elif optimize=='Storage':
     converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
else:    
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
#reduce the size of a floating point model by quantizing the weights to float16
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()
#save the quanitized model toa binary file
open("Seefood_model_V2.tflite", "wb").write(tflite_quant_model)

42732064